#EXCERISE 1

In [ ]:
!pip install -q langchain langchain-ollama langchain-community duckduckgo-search
!curl -fsSL https://ollama.com/install.sh | sh

In [8]:
import os
import subprocess
import time
import threading
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import ChatOllama

In [9]:
def start_ollama():
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

thread = threading.Thread(target=start_ollama)
thread.start()
time.sleep(10)
!ollama pull llama3.2

In [10]:
llm = ChatOllama(model="llama3.2", temperature=0)
tools = [DuckDuckGoSearchRun()]
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [11]:

query = "Who is the current CEO of Twitter (X) and what is his net worth?"
print(f"\n Processing Query: '{query}'\n" + "="*40)

try:
    response = agent_executor.invoke({"input": query})
    print("="*40 + f"\nFINAL ANSWER:\n{response['output']}")
except Exception as e:
    print(f"Error: {e}")


 Processing Query: 'Who is the current CEO of Twitter (X) and what is his net worth?'


> Entering new AgentExecutor chain...
Thought: To find the current CEO of Twitter and their net worth, I should search for recent news articles or official announcements about the company.

Action: duckduckgo_search
Action Input: "Twitter CEO current"Elon Reeve Musk[b] (born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, Twitter , and xAI. In addition to Musk and Twitter , the lawsuit also named former Twitter CEO Jack Dorsey, current Twitter CEO Parag Agrawal and the company’s board as defendants. Musk Won’t Join the Twitter ’s Board#. Just last week, the CEO and co-founder of Tesla confirmed buying a 9.2% stake in Twitter – his most active social platform. What will happen to Twitter current CEO Parag Agrawal is not yet clear. According to CNBC, an SEC filing on Thursday revealed Musk secured approximately... A Twitter user recently posted, "I feel for

#Analysis:

#1. Decomposition: How the LLM breaks down the question
Observation: The agent correctly identified that the user's query consisted of two distinct parts requiring factual retrieval.

Evidence: The initial Thought states: "To find the current CEO of Twitter and their net worth..."

Analysis: The model successfully parsed the compound question. It recognized that it first needed to identify the person (Identity) before it could determine their financial status (Attribute). It did not attempt to guess the name, immediately acknowledging the need for external information.

#2. Tool Selection: When it decides to use the search tool
Observation: The agent chose to use the search tool immediately after receiving the query.

Evidence:

Thought: "...I should search for recent news articles..."

Action: duckduckgo_search

Analysis: This demonstrates correct ReAct behavior. The model recognized that "current CEO" implies dynamic, time-sensitive information that would not be in its pre-trained weights (or might be outdated in its weights).

#3. Tool Input: Examination of the search query
Observation: The agent generated the search query: "Twitter CEO current".

Evidence: Action Input: "Twitter CEO current"

Analysis: The input was a concise keyword search rather than a natural language question. This is generally efficient for search engines. However, the agent repeated the exact same search query twice (in Step 1 and Step 2), which indicates a lack of refinement. When the first search gave mixed results, a more advanced agent might have refined the second query to "Linda Yaccarino net worth" or "Is Parag Agrawal still CEO?".

#4. Observation Integration: Incorporating search results into reasoning
Observation: The search results were "noisy." They contained conflicting timelines:

Historical data (Jack Dorsey, Parag Agrawal fired in 2022).

Recent data (Linda Yaccarino hired in 2023).

Note: The log contains peculiar future dates ("Jul 9, 2025", "Sep 19, 2025") suggesting the local model may have hallucinated parts of the search snippet or the context was set to the future.

Reasoning Flaw:

In Step 2, the agent read about Linda Yaccarino but then regressed, thinking: "I should check if Parag Agrawal is still the CEO..." despite the search result explicitly stating he was fired in 2022.

The agent struggled to filter "current" from "historical" within the same text block.

#5. Formulation of Final Answer
Observation: The final answer is factually mixed and exhibits a logical pivot.

The Answer: "There is no current CEO of Twitter (X)... Linda Yaccarino took over... The net worth of Elon Musk is... $200 billion."

Analysis of Behavior:

Identity Confusion: The agent states "There is no current CEO" but immediately contradicts itself by saying "Linda Yaccarino took over." This suggests it was confused by the rebranding from Twitter to X.

Gender/Pronoun Bias: The user asked "what is his net worth." The agent found a female CEO (Linda Yaccarino) but, likely influenced by the user's pronoun "his," decided to answer with Elon Musk's net worth instead of Linda's.

Conclusion: The agent prioritized the user's linguistic cue ("his") over the factual hierarchy (the CEO is Linda), resulting in a disjointed answer where the CEO is identified as Linda, but the net worth provided is Elon's.

#Excerise-2

In [ ]:
!sudo apt-get install -y pciutils
!pip install -q langchain langchain-ollama langchain-community duckduckgo-search faiss-cpu sentence-transformers langchain_huggingface
!pip install -U ddgs
!curl -fsSL https://ollama.com/install.sh | sh

In [4]:
import os
import subprocess
import time
import threading
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import ChatOllama

In [5]:
def start_ollama():
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

thread = threading.Thread(target=start_ollama)
thread.start()
time.sleep(10)

!ollama pull llama3.2

In [6]:
secret_documents = [
    Document(page_content="Project Chimera is a top-secret AI initiative launched in 2024 by TechCorp."),
    Document(page_content="The lead engineer of Project Chimera is Dr. Elena Vance."),
    Document(page_content="The budget for Project Chimera is 500 million dollars, approved by the board."),
    Document(page_content="Project Chimera aims to integrate quantum computing with biological neural networks.")
]

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(secret_documents, embeddings)
retriever = vector_store.as_retriever()
memory_tool = create_retriever_tool(
    retriever,
    "search_internal_documents",
    "Searches for internal company documents, secrets, and project details. Use this for questions about Project Chimera."
)

In [8]:
llm = ChatOllama(model="llama3.2", temperature=0)
search_tool = DuckDuckGoSearchRun()
tools = [search_tool, memory_tool]
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [9]:
print(" TEST 1: MEMORY QUERY (Internal Knowledge)")
query_memory = "Who is the lead engineer of Project Chimera and what is the budget?"
agent_executor.invoke({"input": query_memory})


print(" TEST 2: WEB QUERY (External Knowledge)")
query_web = "What is the current stock price of Tesla?"
agent_executor.invoke({"input": query_web})

 TEST 1: MEMORY QUERY (Internal Knowledge)


> Entering new AgentExecutor chain...
Question: Who is the lead engineer of Project Chimera and what is the budget?

Action: search_internal_documents
Action Input: {'query': 'Project Chimera lead engineer and budget'}
The budget for Project Chimera is 500 million dollars, approved by the board.

The lead engineer of Project Chimera is Dr. Elena Vance.

Project Chimera is a top-secret AI initiative launched in 2024 by TechCorp.

Project Chimera aims to integrate quantum computing with biological neural networks.Question: Who is the lead engineer of Project Chimera and what is the budget?

Action: search_internal_documents
Action Input: {'query': 'Project Chimera lead engineer and budget'}
The budget for Project Chimera is 500 million dollars, approved by the board.

The lead engineer of Project Chimera is Dr. Elena Vance.

Project Chimera is a top-secret AI initiative launched in 2024 by TechCorp.

Project Chimera aims to integrate quantum c

{'input': 'What is the current stock price of Tesla?',
 'output': 'The current stock price of Tesla is not available as the information provided by duckduckgo_search is outdated. However, according to recent data, the current stock price of Tesla (TSLA) is around $456.56 per share.\n\nNote: The final answer may change rapidly due to market fluctuations and the information provided by duckduckgo_search might be outdated.'}

#Analysis:
#1. Tool Selection Strategy (Routing)
Observation: The agent demonstrated excellent semantic routing. It correctly identified the context of the query to select the appropriate tool without hesitation.

Evidence:

Internal Query: When asked about "Project Chimera" (a fictional entity defined only in the vector store), the agent immediately triggered search_internal_documents. It did not attempt to search the web for this private information.

External Query: When asked about "Tesla stock price" (public, real-time information), the agent immediately triggered duckduckgo_search.

Verdict: The tool descriptions provided in the code (e.g., "Use this for questions about Project Chimera") were highly effective. The specific keyword "Project Chimera" acted as a strong trigger for the local model.

#2. Information Integration (Observation to Reasoning)
Observation (Memory):

The retrieved documents provided clear facts: "budget... 500 million" and "lead engineer... Dr. Elena Vance."

Issue: The agent entered a verification loop. The log shows it performing the same search_internal_documents action multiple times (at least 4-5 times), getting the exact same result, and thinking "I already know this information... but I'll reiterate it for clarity."

Analysis: While it retrieved the correct data, the Llama 3.2 model struggled to trust its own context, resulting in redundant API calls.

Observation (Web):

The web search returned "noisy" data with conflicting dates (snippets mentioning 2020, 2021, and even a hallucinated/future date of "Jan 27, 2026").

Success: Despite the noise, the agent successfully extracted the specific value "$456.56" buried in the text.

Nuance: The agent was skeptical of the data quality, adding a disclaimer in the Final Answer: "The information... might be outdated." This indicates the model has a high degree of "honesty" or safety alignment.

#3. Recommendations for Prompt Improvement
To fix the looping behavior and improve confidence, the following adjustments are recommended:

Fix the Looping (Repetition):

Current Issue: The agent keeps searching for what it already found.

Improvement: Update the system prompt with a "Termination Condition."

Add this to prompt: "If the Observation contains the answer, you MUST output the 'Final Answer' immediately. Do not repeat the same Action."

Fix the "Outdated" Fear:

Current Issue: The agent apologized that web data might be old.

Improvement: Update the duckduckgo_search tool description.

Change to: "A search engine. Use this to find current, real-time events and stock prices. Assume results labeled 'Today' or 'Live' are accurate."

Final Conclusion
The integration of Long-Term Memory was successful. The agent creates a clear boundary between "Secret Knowledge" (Vector Store) and "Public Knowledge" (Web). The primary issue observed was efficiency (looping actions) rather than accuracy.

#CONCLUSION
In this session, we successfully built a robust ReAct (Reason + Act) Agent capable of distinguishing between internal secrets and external real-time data. We started by configuring a local Llama 3.2 model via Ollama to run efficiently on Google Colab using background threading. By leveraging LangChain, we orchestrated the agent's logic, initially giving it "eyes" with the DuckDuckGo Search tool to fetch live information like stock prices. We then evolved the architecture by integrating Long-Term Memory using a FAISS vector store, allowing the agent to recall private "Project Chimera" details without hallucinating or leaking data to the web. The final result was a context-aware agent that demonstrated intelligent routing—automatically deciding whether to query its internal memory or search the internet based on the user's intent, all while maintaining a transparent "Thought → Action → Observation" reasoning loop.